<a href="https://colab.research.google.com/github/shaevitz/MOL518-Intro-to-Data-Analysis/blob/main/Lecture_13/MOL518_Lecture13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In colab run this cell first to setup the file structure!
%cd /content
!rm -rf MOL518-Intro-to-Data-Analysis

!git clone https://github.com/shaevitz/MOL518-Intro-to-Data-Analysis.git
%cd MOL518-Intro-to-Data-Analysis/Lecture_13


# Lecture 13: Curve Fitting Beyond Linear Regression

In this class, we will learn how to fit nonlinear curves to biological datasents and plot them in Python using Jupyter Notebooks running in Google Colab.

**Learning objectives**
- Understand when and why nonlinear models are needed (vs. linearization)
- Learn how to fit nonlinear models to data using `scipy.optimize.curve_fit`
- Interpret parameter estimates and covariance
- Evaluate the curve fits using residuals, $R^2$, and Akaike Information Criterion (AIC)
- Apply curve fitting to common biological models: exponential growth, Michaelis–Menten kinetics, Hill equations

> Tip: Run cells in order. Each section is independent but reuses utility functions defined below.



## Setup & utilities
This cell imports libraries and defines helper functions for metrics and plotting.


In [ ]:
# import necessary libraries: numpy, pandas, matplotlib, scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Matplotlib defaults
plt.rcParams.update({
    "figure.figsize": (6.5, 4.2),
    "axes.grid": True,
    "grid.alpha": 0.25,
})

# Reproducibility
np.random.seed(42) # use the same random seed each time for reproducible results

# ---------- Helper functions ----------

# this function calculates the r^2 value for the curve fit
def r2_score(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res/ss_tot

# this function calculates the Akaike Information Criterion (AIC) for the fit
def aic(y_true, y_pred, k):
    # Akaike Information Criterion for least-squares fits.
    # k = number of free parameters
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)
    rss = np.sum((y_true - y_pred)**2)
    return 2*k + n * np.log(rss / n)

# this function plots the curve fits
def plot_fit(x, y, model, popt, xlabel="x", ylabel="y", title="Fit"):
    x = np.asarray(x)
    y = np.asarray(y)
    xs = np.linspace(np.min(x), np.max(x), 400)
    plt.scatter(x, y, s=32, alpha=0.9, label="Data")
    plt.plot(xs, model(xs, *popt), c="crimson", lw=2.0, label="Model fit")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

# this function will help us plot the residuals after doing curve fitting
def plot_residuals(x, residuals, xlabel="x"):
    plt.axhline(0, color='k', lw=1, alpha=0.6)
    plt.scatter(x, residuals, s=28)
    plt.xlabel(xlabel)
    plt.ylabel("Residuals (y - ŷ)")
    plt.title("Residual plot")
    plt.show()

print("Libraries imported & utilities ready.")



## Motivation: Why nonlinear models?
Many biological relationships are nonlinear:

- **Exponential Growth/decay:** $y = A e^{kt}$ or logistic growth
$y = \frac{K}{1 + e^{-r(t-t_0)}}$

- **Enzyme kinetics (Michaelis–Menten):**
$v = \frac{V_\max [S]}{K_m + [S]}$

- **Cooperative binding (Hill):**
$Y = \frac{[L]^n}{K_d^n + [L]^n}$

Historically, researchers *linearized* these models (e.g., Lineweaver–Burk) but that can *distort errors* and bias estimates. The current best practice is to fit the nonlinear model *directly* to the data by using least squares or maximum likelihood methods.

For the purposes of this class, we will focus on least squares approaches, as they are simpler to teach, and are the method of choice for simple nonlinear curve fitting.

**Note:** We discussed least squares methods in the last class in the context of linear fitting, but it is also really useful for nonlinear fitting! The way the error is calculated is the same.



## Nonlinear regression with `scipy.optimize.curve_fit`
Key ideas:
- Define a Python function `f(x, parameters)` for your model. Typically nonlinear models will have multiple parameters.
- Provide **initial guesses** `p0` (crucial for convergence). If `p0` is not close enough, you may need to rerun the fit with a different value for `p0`.
- Optional **bounds** can constrain parameters (e.g., positivity for concentrations).
- `curve_fit` returns `popt` (best-fit params) and `pcov` (covariance matrix).
- Standard errors are `se = sqrt(diag(pcov))` if the model is well-specified.



## Evaluating fit quality

Once you have made a fit, the next critical step is to determine how good your fit is. There are four key metrics to look at when evaluating a nonlinear curve fit.

- **Residuals:** should be structureless (no systematic biases or trends) and roughly homoscedastic (equal variance across the dataset).
- **$R^2$:** can be thought of as the fraction of variance explained by the fit. Use this metric cautiously for nonlinear models, as $R^2$ is inherently a linear metric. If the dataset is only a little bit nonlinear, $R^2$ might be okay to use sparingly.
- **AIC:** supports model comparison; lower is better for comparable datasets. This is a complicated metric to explain, but feel free to read more about it [here](https://en.wikipedia.org/wiki/Akaike_information_criterion).
- **Parameter uncertainty:** use `pcov` → standard errors; consider calculating confidence intervals using bootstrapping.


## Example 1: Exponential growth

For this very simple example, we will be using synthetic data, generated by adding random, normally distributed noise to the data.

Synthetic data: $OD_{600} = A e^{kt} + \epsilon$

Here, $\epsilon$ denotes normally distributed noise that we are adding to the data to simlulate measurment error in experimental data.


In [ ]:

# ----- Generate synthetic data -----
t = np.linspace(0, 10, 50)
A_true, k_true = 0.03, 0.40
OD600 = A_true * np.exp(k_true * t) + np.random.normal(0, 0.07, size=t.size)

# ----- Model -----
def exp_model(t, A, k):
    return A * np.exp(k * t)

# ----- Fit -----
pnot = [1.0, 0.1]  # initial guesses
popt, pcov = curve_fit(exp_model, t, OD600, p0=pnot)
A_fit, k_fit = popt # this unpacks popt, which is the output of the curve fitting function
perr = np.sqrt(np.diag(pcov))

print(f"A = {A_fit:.3f} ± {perr[0]:.3f}")
print(f"k = {k_fit:.3f} ± {perr[1]:.3f}")

# ----- Evaluate -----
OD600hat = exp_model(t, *popt)
print(f"R^2 = {r2_score(OD600, OD600hat):.3f}")
print(f"AIC = {aic(OD600, OD600hat, k=2):.2f}")

# ----- Plot -----
plot_fit(t, OD600, exp_model, popt, xlabel="Time (hours)", ylabel=r'$OD_{600}$', title="Exponential growth fit")
residuals = OD600 - OD600hat
plot_residuals(t, residuals, xlabel="Time (hours)")


### Questions related to Example 1

1. How well does the model fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. How much variation is there in the parameter estimates?

**[Your answer goes here]**

3. What do the residuals look like? Are structureless and roughly homoscedastic?

**[Your answer goes here]**


## Exercise 1: Michaelis–Menten (enzyme kinetics)

I'm sure you are all at least somewhat familiar with the Michaelis-Menten equations from your undergraduate studies. What you may not have been taught is that they were the first example of quantitative modeling in biology, and that they were figured out long before we had any ideas about how enzymes actually worked **(back in 1913!!)**. For more on this fascinating history, check out [this link](https://www.sciencedirect.com/science/article/pii/S2213020914000627).

For this exercise, I have left the "Model" part blank. You will need to define a function ```mm_model``` using the formula below:

Model: $v = \dfrac{V_{\max} [S]}{K_m + [S]}$.

Parameters:
- $V_{\max}$: maximum reaction velocity
- $K_m$: substrate concentration at half-max velocity

Please do not use generative AI for this exercise!

In [ ]:

# ----- Generate synthetic data -----
S = np.linspace(0.05, 10.0, 30)
Vmax_true, Km_true = 5.0, 2.0
v = Vmax_true * S / (Km_true + S) + np.random.normal(0, 0.4, size=S.size)

# ----- Model -----


# ----- Fit -----
p0 = [1.0, 1.0]
popt, pcov = curve_fit(mm_model, S, v, p0=p0, bounds=(0, np.inf))
Vmax_fit, Km_fit = popt
perr = np.sqrt(np.diag(pcov))

print(f"Vmax = {Vmax_fit:.3f} ± {perr[0]:.3f}")
print(f"Km   = {Km_fit:.3f} ± {perr[1]:.3f}")

# ----- Evaluate & plot -----
vhat = mm_model(S, *popt)
print(f"R^2 = {r2_score(v, vhat):.3f}")
print(f"AIC = {aic(v, vhat, k=2):.2f}")
plot_fit(S, v, mm_model, popt, xlabel="[S] (pM)", ylabel="v (nM/s)", title="Michaelis–Menten fit")
plot_residuals(S, v - vhat, xlabel="[S] (pM)")


### Questions related to Exercise 1

1. How well does the model fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. How much variation is there in the parameter estimates?

**[Your answer goes here]**

3. What do the residuals look like? Are structureless and roughly homoscedastic?

**[Your answer goes here]**


## Excercise 2: Hill equation (ligand binding)

I expect that some of you will have heard of the Hill equation, which is a simple model for cooperative (or non-cooperative, or negatively cooperative) binding between a ligand and a macromolecule (e.g., a protein or nucleic acid).

For this exercise, I have left both the Model and the Fit parts blank for you to fill in as an exercise. You will need to define a model ```hill``` as well as write code to do the curve fitting.

Model: $Y = \dfrac{[L]^n}{K_d^n + [L]^n}$, where $n$ is the Hill (cooperativity) coefficient.

- $K_d$: apparent dissociation constant (ligand at half-occupancy)
- $n>1$: positive cooperativity; $n=1$: noncooperative; $n<1$: negative cooperativity

Please do not use generative AI for this exercise!


In [ ]:

# ----- Generate synthetic data -----
L = np.logspace(-2, 2, 40)
Kd_true, n_true = 5.0, 2.0
Y = L**n_true / (Kd_true**n_true + L**n_true) + np.random.normal(0, 0.05, size=L.size)

# ----- Model -----



# ----- Fit -----


# ----- Evaluate & plot -----
Yhat = hill(L, *popt)
print(f"R^2 = {r2_score(Y, Yhat):.3f}")
print(f"AIC = {aic(Y, Yhat, k=2):.2f}")
plt.semilogx()
plot_fit(L, Y, hill, popt, xlabel="[L] (nM)", ylabel="Y (fraction bound)", title="Hill binding curve fit")
plot_residuals(L, Y - Yhat, xlabel="[L] (nM)")


### Questions related to Exercise 2

1. How well does the model fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. How much variation is there in the parameter estimates?

**[Your answer goes here]**

3. What do the residuals look like? Are structureless and roughly homoscedastic?

**[Your answer goes here]**

## Model Selection: What if I don't know which curve to fit to my data?

In some cases, life is simple. For example, we expect bacteria to undergo exponential growth; enzymes to have Michaelis-Menten kinetics; ligands to bind based on the Hill equation. Deviations from these expectations are interesting as they suggest that additional regulation is going on!

There is no one-size-fits-all approach to model selection, the best approach is to try several possible models and see which one is best. Here are some key criteria to use:

1. **Goodness of fit.** We discussed this above, it is typically measured using $R^2$, or the sum of squared error.

2. **Model complexity and parsimony.** Parsimony means to favor simpler models that achieve a good fit, based on the principle of Occam's Razor. Cross-validation is also a helpful way to make sure your model is general enough and not just overfit to your data.

3. **Interpretability.** It is good to start with models that align with the underlying theory. It is especially helpful if the model has parameters that are interpretable (e.g., doulbing time for exponential growth, or $K_M$ for Michaelis-Menten, or the Hill coefficient).

4. **Residual analysis.** As discussed earlier, ideally the residuals shold be normally distributed with a mean of zero. There shouldn't be any obvious patterns to the residuals.

5. **Outliers.** Sometimes we encounter outliers in our data. It's important to figure out if they are due to errors, or if they are genuine observations that are very different from the norm.

6. **Practical considerations.** Some algorithms may be impractical to run depending on your computational resources and the size of the dataset.

AIC, mentioned above, integrates goodness of fit and model complexity, so is often a good choice to prevent overfitting by penalizing models with more parameters.

## Example 2: Gentamycin data

We will use the file `data/ecoli_drugs.csv`, which has *E. coli* growth curve data in the presence of antibiotics from the Gitai Lab . I've placed a copy of this file in the `data` folder.

You should see that there are 13 columns with the first row as a header to tell you what is in each column. Time is the first column (in minutes, hence the name `Time_min`) and the columns 2-13 show OD in the presence of the different drugs.

In [ ]:
# import statements
import numpy as np

# Load the data, skipping the header row
data = np.loadtxt('data/ecoli_drugs.csv', delimiter=',', skiprows=1)

time_min = data[:, 0] #Time in minutes is the first column

Rifampicin = data[:, 1] # OD for Rifamipicin is the second column
Novabiocin = data[:, 2] # OD for Novabiocin is the third column ...
Trimethoprim = data[:, 3]
Chloramphenicol = data[:, 4]
Ampicillin = data[:, 5]
Gentamycin = data[:, 6]
Gentamycin2 = data[:, 7]
Ampicillin2 = data[:, 8]
Chloramphenicol2 = data[:, 9]
Trimethoprim2 = data[:, 10]
Novabiocin2 = data[:, 11]
Rifampicin2 = data[:, 12]

# ----- Model -----
def exp_model(t, A, k):
    return A * np.exp(k * t)

def logistic_model(t, K, r, t0):
    return K / (1 + np.exp(-r * (t - t0)))

# ----- Fit -----
pnot = [0.01, 0.01]  # initial guesses
popt, pcov = curve_fit(exp_model, time_min, Gentamycin, p0=pnot)
A_fit, k_fit = popt # this unpacks popt, which is the output of the curve fitting function
perr = np.sqrt(np.diag(pcov))

print(f"A = {A_fit:.3f} ± {perr[0]:.3f}")
print(f"k = {k_fit:.3f} ± {perr[1]:.3f}")

# ----- Evaluate -----
Gent_hat = exp_model(time_min, *popt)
print(f"R^2 = {r2_score(Gentamycin, Gent_hat):.3f}")
print(f"AIC = {aic(Gentamycin, Gent_hat, k=2):.2f}")

# ----- Plot -----
plot_fit(time_min, Gentamycin, exp_model, popt, xlabel="Time (hours)", ylabel=r'$OD_{600}$', title="Exponential growth fit")
residuals = Gentamycin - Gent_hat
plot_residuals(time_min, residuals, xlabel="Time (hours)")

### Questions associated with Example 2

1. How well does the model fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. How much variation is there in the parameter estimates?

**[Your answer goes here]**

3. What do the residuals look like? Are structureless and roughly homoscedastic?

**[Your answer goes here]**

## Exercise 3: Logistic fitting to Gentamycin data

Fit a logistic curve to the Gentamycin data.

Please do not use generative AI for this exercise!

In [ ]:
# your code for Exercise 3 goes here!



### Questions associated with Exercise 3

1. How well does the model fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. What do the residuals look like? Are structureless and roughly homoscedastic?

**[Your answer goes here]**

3. What is missing from the models we are fitting to the data? Are there any biological insights to be gained from this?

**[Your answer goes here]**

## Exercise 4: Novabiocin data

Fit exponential and logistic curves to the Gentamycin data.

Please do not use generative AI for this exercise!

In [ ]:
# Your code for Exercise 4 goes here!



### Questions related to Exercise 4

1. How well does the model fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. What do the residuals look like? Are structureless and roughly homoscedastic?

**[Your answer goes here]**

3. What is missing from the models we are fitting to the data? Are there any biological insights to be gained from this?

**[Your answer goes here]**

# Bonus exercises

If you have time, or would like extra practice after class, please complete the following exercises. Please note that they are a bit harder than the regular exercises (that is on purpose!).

## Bonus Exercise 1

Try to fit a logistic curve to the Chloramphenicol data. Also try to fit a double exponential curve to the data


In [ ]:
# Your code for Bonus Exercise 1 goes here!



### Questions related to Bonus Exercise 1

1. How well do each of the models fit the data (based on visual inspection of the graph)?

**[Your answer goes here]**

2. Why do you think this is the case? Try to come up with a biological explanation for this.

**[Your answer goes here]**

## Bonus Exercise 2

Try to come up with a simple model that fits the Gentamycin data even better than the logistic model.

In [ ]:
# Your code for Bonus Exercise 2 goes here!



### Questions related to Bonus Exercise 2

1. What models did you explore and why?

**[Your answer goes here]**

2. What biological process(es) are your models accounting for?

**[Your answer goes here]**